# 학습 관련 정보 저장하기
인공 신경망 학습은 그 크기에 따라 몇 초 혹은 며칠이 걸릴 수 있는 작업이다. 따라서 작업이 길어질 경우 관련 정보를 중간에 저장해야만 같은 작업을 반복하지 않을 수 있다. 또한 정전, 인터넷 장애 등과 같은 예상치 못한 일로 작업을 도중에 중단할 경우가 발생하기도 한다. 이 때 저장된 정보를 활용하여 다시 중단한 시점부터 학습을 재개할 수 있다.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd/content/gdrive/MyDrive/파이토치/딥러닝호형

/content/gdrive/MyDrive/파이토치/딥러닝호형


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

## 1) 데이터 불러오기

In [ ]:
df = pd.read_csv('./data/reg.csv', index_col=[0])

## 2) 데이터 변수와 타겟값 나누기

In [ ]:
X = df.drop('Price', axis=1).to_numpy()
Y = df['Price'].to_numpy().reshape((-1, 1))

## 3) 텐서 데이터와 배치 만들기

In [ ]:
class TensorData(Dataset):

  def __init__(self, x_data, y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.FloatTensor(y_data)
    self.len = self.y_data.shape[0]

  def __getitem__(self, index):

    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, tset_size=0.5)

trainsets = TensorData(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(trainsets, batch_size=32, shuffle=True)

testsets = TensorData(X_test, Y_test)
testloader = torch.utils.data.DataLoader(testsets, batch_size=32, shuffle=False)

## 4) 모델 구축
모델은 Regressor로 정의하며 입력층(노드13개), 2개의 은닉층(50개, 30개), 출력층(1개)으로 구성한다. 데이터의 변수는 13개이므로 입력층의 노드는 13개가 되고, 출력층은 집 값 단일 값을 추출하는 것이기에 1개가 된다. 은닉층에 대해서는 실험을 하면서 튜닝을 할 수 있다.

In [ ]:
class Regressor(nn.Module):
  def __init__(self):
    super.__init__() # 모델 연산 정의
    self.fc1 = nn.Linear(13, 50, bias=True) # 입력층(13)->은닉층1(50)으로 가는 연산
    self.fc2 = nn.Linear(50, 30, bias=True) # 입력층1(50)->은닉2(30)으로 가는 연산
    self.fc3 = nn.Linear(30, 1, bias=True) # 입력층2(30)->출력층(1)으로 가는 연산
    self.dropout = nn.Dropout(0.2) # 연산이 될 때마다 20%의 비율로 랜덤하게 노드를 없앤다.

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.dropout(F.relu(self.fc2(x)))
    x = F.relu(self.fc3(x))

    return x

# 주의할 점은 dropout은 과적합 방지를 위한 노드 제거이므로, 출력층에서는 절대 사용X

## 5) 모델, 손실함수, 최적화 방법 선언

In [ ]:
model = Regressor()
criterion = nn.MSELoss()

# lr은 학습률
# weight_decay는 L2 정규화에서의 penalty 정도를 의미
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)

In [ ]:
pret = 0

if pret == 1: # pretrain 모델이 있을 때는 pret를 1로 해서 아래를 실행하여 변수 선언(값 할당)
  checkpoint = torch.load('./model/4-4reg.pt')
  model.load_state_dict(checkpoint['model'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  loss = checkpoint['loss']
  ep = checkpoint['ep']
  ls = loss_[-1]
  print(f'epoch={ep}, loss={ls}')
  ep = ep + 1

else: # predtrain 모델이 없을 때
  ep = 0 # 에폭 0부터 시작
  ls = 1 # ls 1미만이면 모델 저장
  loss_ = [] # 그래프를 그리기 위한 loss 저장용 리스트

## 6) 학습 진행

In [ ]:
n = len(trainloader)

for epoch in range(ep, 2000):

  running_loss = 0.0

  for i, data in enumerate(trainloader, 0): # 무작위로 섞인 32개 데이터가 있는 배치가 하나씩 들어온다.
    inputs, values = data # data에는 X, Y가 들어있다.

    optimize.zero_grad() # 최적화 초기화

    outptus = model(inputs) # 모델에 입력값 대입 후 예측값 산출
    loss = criterion(outputs, values) # 손실 함수 계산
    loss.backward() # 손실 함수 기준으로 역전파 실행
    optimizer.step() # 역전파를 진행하고 가중치 업데이트

    running_loss += loss.item() # epoch 마다 평균 loss를 계산하기 위해 배치 loss를 더한다.

  l = running_loss/n
  loss_.append(l) # MSE(Mena Squared Error) 계산
  if l < ls: # 모델 저장
    torch.save({'epoch' : epoch,
                'loss' : loss_,
                'model' : model.state_dict(),
                'optimizer': optimizer.state_dict()
                },
               './models/4-4reg.pt') # dict를 통해 여러가지 정보를 저장할 수 있다!

print('Finished Training')

In [ ]:
len(loss_)

In [ ]:
plt.plot(loss_)
plt.title('Training Loss')
plt.xlabel('epoch')
plt.show()

## 7) 모델 평가

In [ ]:
def evaluate(dataloader):

  predictions = torch.tensor([], dtype=torch.float) # 예측값을 저장하는 텐서
  actual = torch.tensor([], dtype=torch.float) # 실제값을 저장하는 텐서

  with torch.no_grad():
    model.eval() # 평가를 할 때에는 .eval() 반드시 사용해야 한다.
    for data in dataloader:
      inputs, values = data

      predictions = torch.cat((predictions, outputs), 0) # cat을 통해 예측값을 누적
      actual = torch.cat((actual, values), 0) # cat을 통해 실제값을 누적
  rmse = np.sqrt(mean_squared_error(predictions, actual)) # sklearn을 이용한 rmse계산

  return rmse

# 평가 시 .eval()을 사용해야 하는 이유
# 평가 시에는 온전한 모델로 평가를 해야 하는데 .eval()이 아닌 .train()인 경우 드롭아웃이 활성화되어 있다.
# 따라서 드롭아웃이나 배치 정규화 등과같이 학습 시에만 사용하는 기술들을 평가 시에는 비활성화해야 한다.

In [ ]:
cheackpoint = torch.load('./models/4-4reg.pt')
model.load_state_dict(checkpoint['model'])

In [ ]:
train_rmse = evaluation(trainloader) # 학습 데이터의 rmse
test_rmse = evaluation(testloader) # test data 의 rmse

print(f'Train RMSE : {train_rmse}')
print(f'Test RMSE : {test_rmse}')

# 예시를 위한 단순한 비교일 뿐